In [120]:
import pandas as pd
import json

In [121]:
fault_names = pd.read_csv('fault_names.csv')
#use overrideName, otherwise fullName if empty
fault_names['name'] = fault_names['overrideName'].fillna(fault_names['fullName'])
#remove if include=false
fault_names = fault_names[fault_names['include'] == True]

In [122]:
df_centres = pd.read_csv('Driving Test Centres.csv')
df_centres = df_centres.set_index('id')
df_centres = df_centres.fillna('')

In [123]:
#ignore the first row. use second row as header
df_annexc = pd.read_csv('Annex C.csv', skiprows=2, thousands=',')
df_annexc = df_annexc.set_index('Test ID')


In [ ]:
df_annexb = pd.read_csv('Annex B.csv', skiprows=2, thousands=',')
#keep cols Test ID, SLOT_TIME, ATTEMPTS
df_annexb = df_annexb[['Test ID', 'SLOT_TIME', 'ATTEMPTS']]
df_annexb = df_annexb.set_index('Test ID')

df_all = df_annexc.join(df_annexb, on='Test ID')

In [ ]:
# remove comma from 'TC_ID' column values
df_all['TC_ID'] = df_all['TC_ID'].astype(int)
df_all['pass'] = df_all['TEST_RESULT'].apply(lambda x: 1 if x == 'P' else 0)
df_all['SLOT_TIME'] = pd.to_datetime(df_all['SLOT_TIME'], format='%H:%M')
df_all['TIME_BIN'] = df_all['SLOT_TIME'].dt.floor('60T')
df_all['TIME_BIN_STR'] = df_all['TIME_BIN'].dt.strftime('%H:%M') + '-' + (df_all['TIME_BIN'] + pd.Timedelta('1H')).dt.strftime('%H:%M')
df_all['PERIOD_DATE'] = pd.to_datetime(df_all['PERIOD_DATE'], format='%d/%m/%Y')
df_all['DAY_OF_WEEK'] = df_all['PERIOD_DATE'].dt.day_name()
df_all['DAY_TYPE'] = df_all['PERIOD_DATE'].apply(lambda x: 'Mon-Fri' if x.weekday() < 5 else 'Saturday' if x.weekday() == 5 else 'Sunday')
df_all['ATTEMPTS_TRUNC'] = df_all['ATTEMPTS'].apply(lambda x: '5+' if x >= 5 else str(x))
df_all['FIRST_ATTEMPT'] = df_all['ATTEMPTS'].apply(lambda x: 1 if x == 1 else 0)

In [ ]:
# any columns that end in _DANGER? Rename them to _DANGEROUS
danger_columns = [col for col in df_all.columns if col.endswith('_DANGER')]
for col in danger_columns:
    df_all[col.replace('_DANGER', '_DANGEROUS')] = df_all[col]
    df_all = df_all.drop(columns=[col])

In [ ]:
maneuvres = pd.read_csv('maneuvre_names.csv')
#create dict of databaseField to shortName
reverse_maneuvres = maneuvres[maneuvres['isReverseManeuvre'] == True].set_index('databaseField')['shortName'].to_dict()

In [ ]:

def get_maneuvre_code(row):
    for databaseField in reverse_maneuvres:
        if row[databaseField] == 1:
            return reverse_maneuvres[databaseField]
    return 'Reversing maneuvre skipped'

df_all['reverse_maneuvre'] = df_all.apply(get_maneuvre_code, axis=1)

In [ ]:
# it is possible though very rare for both a serious and dangerous fault to be recorded for the same test.
# The dangerous/serious/total columns seem to be completely independent, although I thought minors should be disregarded if a serious or dangerous fault is present.
# I will add suffix _FAILS to mean the max of both. _TOTAL actually means _MINOR
for index, row in fault_names.iterrows():
    #create fields if they don't exist
    prefix = row['databaseFieldPrefix']
    if prefix+ '_SERIOUS' not in df_all:
        df_all[prefix + '_SERIOUS'] = 0
    if prefix + '_DANGEROUS' not in df_all:
        df_all[prefix + '_DANGEROUS'] = 0
    if prefix + '_TOTAL' not in df_all:
        df_all[prefix + '_TOTAL'] = 0
    # serious or dangerous can only be 0 or 1. Take OR of both
    df_all[prefix + '_FAILS'] = df_all[[prefix + '_SERIOUS', prefix + '_DANGEROUS']].any(axis=1).astype(int)

In [ ]:
for index, row in maneuvres.iterrows():
    man_code = row['databaseField']
    fault_field_prefix = row['faultFieldPrefix']
    cols = [col for col in df_all.columns if col.startswith(fault_field_prefix) and col.endswith('_FAILS')]
    df_all[man_code + '_FAILS'] = df_all[cols].any(axis=1).astype(int)

In [ ]:
dangerous_faults_array = [f + '_DANGEROUS' for f in fault_names['databaseFieldPrefix']]
serious_faults_array = [f + '_SERIOUS' for f in fault_names['databaseFieldPrefix']]
minor_faults_array = [f + '_TOTAL' for f in fault_names['databaseFieldPrefix']]
df_all['dangerous_count'] = df_all[dangerous_faults_array].sum(axis=1)
df_all['serious_count'] = df_all[serious_faults_array].sum(axis=1)
df_all['minor_count'] = df_all[minor_faults_array].sum(axis=1)
df_all['dangerous_any'] = df_all[dangerous_faults_array].any(axis=1).astype(int)
df_all['serious_any'] = df_all[serious_faults_array].any(axis=1).astype(int)
df_all['minor_any'] = df_all[minor_faults_array].any(axis=1).astype(int)

In [ ]:
def process_aggregation(df_filtered, identifier):
    df_avgs = df_filtered.mean(axis=0, numeric_only=True)
    obj = {'minors': {}, 'fails': {}, 'maneuvres': []}
    for index, row in fault_names.iterrows():
        fault_name = row['name']
        obj['minors'][fault_name] = df_avgs[row['databaseFieldPrefix'] + '_TOTAL']
        obj['fails'][fault_name] = df_avgs[row['databaseFieldPrefix'] + '_FAILS']
    obj['totalTestCount'] = len(df_filtered)
    obj['dailyTestCount'] = obj['totalTestCount'] / 215
    obj['pass'] = df_avgs['pass']
    obj['dangerous_count'] = df_avgs['dangerous_count']
    obj['serious_count'] = df_avgs['serious_count']
    obj['minor_count'] = df_avgs['minor_count']
    obj['dangerous_any'] = df_avgs['dangerous_any']
    obj['serious_any'] = df_avgs['serious_any']
    obj['minor_any'] = df_avgs['minor_any']
    obj['first_attempt'] = df_avgs['FIRST_ATTEMPT']
    obj['attempts'] = df_avgs['ATTEMPTS']


    for code, man_name in maneuvres.set_index('databaseField')['shortName'].items():
        df_filt_man = df_filtered[df_filtered[code] == 1]
        pass_rate = df_filt_man.mean(axis=0, numeric_only=True)['pass']
        pass_rate = 1.0 if pd.isna(pass_rate) else pass_rate
        maneuvre_fails = df_filt_man.mean(axis=0, numeric_only=True)[code + '_FAILS']
        maneuvre_fails = 0 if pd.isna(maneuvre_fails) else maneuvre_fails
        obj['maneuvres'].append({'name': man_name, 'frequency': df_avgs[code], 'pass': pass_rate, 'maneuvre_fails': maneuvre_fails})


    df_times = df_filtered.groupby(['DAY_TYPE', 'TIME_BIN_STR']).agg({'pass': 'mean', 'TC_ID': 'count'}).reset_index()
    df_times['dailyTests'] = (df_times['TC_ID'] / 215) * 7
    df_times['dailyTests'] = df_times.apply(lambda x: x['dailyTests'] / 5 if x['DAY_TYPE'] == 'Mon-Fri' else x['dailyTests'], axis=1)
    df_times = df_times.rename(columns={'DAY_TYPE': 'dayType', 'TIME_BIN_STR': 'time'})
    df_times = df_times.drop(columns=['TC_ID'])
    obj['times'] = df_times.to_dict(orient='records')
    df_days = df_filtered.groupby(['DAY_TYPE']).agg({'pass': 'mean', 'TC_ID': 'count'}).reset_index()
    df_days['dailyTests'] = (df_days['TC_ID'] / 215) * 7
    df_days['dailyTests'] = df_days.apply(lambda x: x['dailyTests'] / 5 if x['DAY_TYPE'] == 'Mon-Fri' else x['dailyTests'], axis=1)
    df_days = df_days.rename(columns={'DAY_TYPE': 'dayType'})
    df_days = df_days.drop(columns=['TC_ID'])
    df_days['time'] = 'All times'
    obj['times'] = obj['times'] + df_days.to_dict(orient='records')

    obj['name'] = str(identifier)
    if obj['name'].isnumeric():
        obj['id'] = int(identifier)
        if identifier in df_centres.index:
            obj['address'] = df_centres.loc[identifier].to_dict()
            obj['name'] = obj['address']['name']

    with open('dtc_data/' + str(identifier) + '.json', 'w') as f:
        json.dump(obj, f, indent=4)
    

In [ ]:
process_aggregation(df_all, 'national')

In [ ]:
for maneuvre in maneuvres['databaseField']:
    process_aggregation(df_all[df_all[maneuvre] == 1], maneuvre)


In [ ]:
for id in df_all['TC_ID'].unique():
    process_aggregation(df_all[df_all['TC_ID'] == id], id)

In [ ]:
for dow in df_all['DAY_TYPE'].unique():
    process_aggregation(df_all[df_all['DAY_TYPE'] == dow], dow)
for time_bin in df_all['TIME_BIN_STR'].unique():
    process_aggregation(df_all[df_all['TIME_BIN_STR'] == time_bin], time_bin.replace(':', ''))

In [ ]:
#print tests that had serious_any but pass=1 YIELDS NONE


# print tests by minor count - show pass rate and test count INTERESTING - SHOULD SHARE ON MISC PAGE
df_all.groupby('minor_count').agg({'pass': ['mean', 'count']})

# print tests by dangerous count - show pass rate and test count INTERESTING - SHOULD SHARE ON MISC PAGE
df_all.groupby('serious_count').agg({'pass': ['mean', 'count']})


unusual_fails = df_all[(df_all['serious_any'] == 0) & (df_all['dangerous_any'] == 0) & (df_all['minor_count'] <= 15) & (df_all['pass'] == 0)]
if len(unusual_fails) > 0:
    print('WARNING: There are unusual fails with no S+Ds')
    print(unusual_fails.head())

unusual_passes = df_all[((df_all['serious_any'] > 0) | (df_all['dangerous_any'] > 0) | (df_all['minor_count'] > 15)) & (df_all['pass'] == 1)]

if len(unusual_passes) > 0:
    print('WARNING: There are unusual passes, those passing despite S+Ds')
    print(unusual_passes.head())